# RNN/LSTM using keras

To further improve upon the vanilla RNN notebook I've here tried to obtained better results using a an LSTM in keras. 
GPU is used to speed up the training process. 

I have not done much analysis on good network structures or hyperparameters, at the moment a 3 layer network with dropout is used. Trained for a couple of hours.

In [2]:
# Imports
import numpy as np
import pandas as pd
import os
import pylab as pb
import tensorflow as tf
import keras

## Classes and functions

In [21]:
def to_one_hot(chars, mapping):
    values = [];
    for c in chars:
        values.append(mapping[c])
    values = np.array(values)
    # Remove }, only occurs ones and causes a bug atm
    n_values = mapping[max(mapping, key=mapping.get)]
    return (np.eye(n_values+1)[values])

def generate_text(model, seq_length, nr_chars, ind_to_char, random_draw = False):
    # Get random starting character ind
    #ind = [np.random.randint(nr_chars)]
    ind= [61]
    
    # Convert to character and create container for results
    text = [ind_to_char[ind[-1]]]
    X = np.zeros((1, seq_length, nr_chars))
    
    # Loop over the sequence length and generate letters
    for i in range(seq_length):
        # Get one-hot repr
        X[0, i, :][ind[-1]] = 1
        #print(ind_to_char[ind[-1]], end="")
        
        # Get prediction and convert to character,
        # either by random draw or the most probable
        if(random_draw):
            ind_distr = model.predict(X[:, :i+1, :])[0,i]
            ind = np.random.choice(nr_chars, 1, p=ind_distr.ravel())
        else:
            ind = np.argmax(model.predict(X[:, :i+1, :])[0], 1)
        text.append(ind_to_char[ind[-1]])
        
    return ('').join(text)

## Load and clean data

In [4]:
# Load Book
book_path = os.getcwd() + "\data\goblet_book.txt"
book = np.loadtxt(book_path,delimiter="%c",dtype="str")
book_data = ''.join(book)

# Get unique characters and create mappings from characters to numbers
char_to_ind = {c: i for i, c in enumerate(reversed(book_data))}
book_chars = np.array(list(char_to_ind))
ind_to_char = {}

# Switch key in the mappings to number between 0-len(book_chars)
for i in range(0,len(book_chars)):
    char_to_ind[book_chars[i]]=i;
    ind_to_char[i] = book_chars[i];
    
# Save the mappings
#np.save('char_to_ind.npy', char_to_ind) 
#np.save('ind_to_char.npy', ind_to_char) 

## Set hyperparameters and set up the data-set

If loading an old model, load mappings first

In [5]:
# Load previous mappings
char_to_ind = np.load('models\mappings\char_to_ind.npy').item()
ind_to_char = np.load('models\mappings\ind_to_char.npy').item()

In [6]:
# Set some hyper params
seq_length = 25
nr_chars   = len(book_chars)
hidden_dim = 500;
dropout_rate = 0.3


# Create and fill containers for all the data
X = np.zeros((int(len(book_data)/seq_length), seq_length, nr_chars))
Y = np.zeros((int(len(book_data)/seq_length), seq_length, nr_chars))

for i in range(X.shape[0]):
    # Get a sequence of text from the book
    X_seq = book_data[i*seq_length   : (i+1)*seq_length  ]
    Y_seq = book_data[i*seq_length+1 : (i+1)*seq_length+1]
    
    # get One-Hot representation and save in containers
    X_hot = to_one_hot(X_seq, char_to_ind)
    Y_hot = to_one_hot(Y_seq, char_to_ind)
    X[i,:,:] = X_hot
    Y[i,:,:] = Y_hot



# Build model

Skip this if loading a model

In [ ]:
# Create model
my_model = keras.Sequential()

# Add LSTM layer
my_model.add(keras.layers.cudnn_recurrent.CuDNNLSTM(hidden_dim, input_shape=(None, nr_chars), return_sequences=True))

# Add drop-out
my_model.add(keras.layers.Dropout(dropout_rate))

# Add more LSTM layers of the same size
my_model.add(keras.layers.cudnn_recurrent.CuDNNLSTM(hidden_dim, input_shape=(None, nr_chars), return_sequences=True))
my_model.add(keras.layers.cudnn_recurrent.CuDNNLSTM(hidden_dim, input_shape=(None, nr_chars), return_sequences=True))

# Change return type to return a matrix and not 3D matrix, now returns a (seq_length,nr_chars) matrix
my_model.add(keras.layers.TimeDistributed(keras.layers.Dense(nr_chars)))

# Add softmax activation, cross-entroyp for categorical and AdaGrad
my_model.add(keras.layers.Activation('softmax'))
my_model.compile(loss="categorical_crossentropy", optimizer="AdaGrad")

## Load Model

Skip if building a new model

In [7]:
# Load models and mappings
my_model = keras.models.load_model('models\Trained_Model.h5')

## Train model

In [ ]:
counter = 12

# Train until stopped. Save model every 10 epochs.
while True:
    print('\n')
    my_model.fit(X, Y, batch_size=50, verbose=1, epochs=5)
    counter += 1
    if counter % 2 == 0:
        # Generate sample text along the way
        generate_text(my_model, 200, nr_chars, ind_to_char)
        my_model.save('model_at_epoch_{}.h5'.format(5*counter))

## Some synthezised texts after training

*Krum to Mr. Weasley tensely.Bagman seemed to knew his mouth was twitched up - but the remains were moving chocolallieass and their paddock feeling that Saturn was fast is a bit chilly without without being friends with their wands on they dressed in front of the pink, particularly sating up in forging himself to sit a right to the three approach with the ground with them and sat down on the narrow beard magerantly.*

*Harry and Ron loudly as though he was starting to seed up with his shoulder. He even had to get the Future, and one of them speaking against the hall, which was spont for a second thingy we not when we were to be being not everything, but you're going to laugh, grounds will send to watch Dumbledore, or it.  It's still thinking about what as coming with middling a sickntic of student that looked like almost at once, so that he could see that he tried to attack him.  He put it back into the box.*

*Voldemort could still have yon he is worked for a simple more among that about Hagrid.""You don't think it they'll bike my son anything - even keep on top of Dudley.  She was making his close friend, and Harry was as time was still by a father burst funion, maybe had heard it for anything to do something to - look at Hogwarts, sleeping and shook their hand.  He turned to look at the bottom of the path, and recoiled softly whispering from dark hair.  He examed a point of the skrewts.*

*!" Harry yelled, pointing at it, right nunty to look at her, and that he never even to dons whether all inside."Harry Potter was now enclosed in the summer holidays.  He waited for the summer would have been in the fourth year and a funny grow difuce-effles outraged, but all the same place where Harry had wanted her woman while you and your mother could not have been able to get through years.""Well, thats what Diggory?" said Mr. Crouch sharply seemed to wanted to the right behind their things*